## Harmony Py Library
### Job Results Example

In [ ]:
import sys
sys.path.append('..')
from harmony import BBox, Client, Collection, Request


Cloud access credentials can be retrieved from an instantiated Client.

In [ ]:
harmony_client = Client()  # assumes .netrc usage

creds = harmony_client.aws_credentials()


Then, instantiate a boto3 client with those credentials.

In [ ]:
#
# NOTE: Execution of this cell will only succeed within the AWS us-west-2 region. 
#

import boto3

s3 = boto3.client('s3', **creds)
uri = 's3://harmony-uat-staging/public/harmony/netcdf-to-zarr/817a3e99-d53c-4169-b9f1-82cc947793be/2020_01_01_7f00ff_global.zarr'
bucket_name = uri.split('/')[2]
object_name = '/'.join(uri.split('/')[3:])
file_name = uri.split('/')[-1]

with open(file_name, 'wb') as f:
    # should return a 403 Forbidden if run outside of us-west-2
    s3.download_fileobj(bucket_name, object_name, f)

